# Mosaic による地理空間データの分散処理

このエクササイズでは、**行ごとにポリゴン情報が含まれる** [GeoJSONSeq](https://gdal.org/drivers/vector/geojsonseq.html) ファイルを Spark でロードし、各ワーカー上で分散処理を行います。

[Mosaic](https://databrickslabs.github.io/mosaic/index.html) は、非常に大きな地理空間データセットを簡単かつ高速に処理できるようにするApache Sparkフレームワークの拡張機能です。

Mosaic は以下を提供します：
- 一般的な空間データ エンコーディング (WKT、WKB、および GeoJSON) 間の簡単な変換。
- Spark ネイティブ データ型から新しいジオメトリを簡単に生成するコンストラクター。
- 空間データセットを変換、集約、および結合するための Spark Expressions として実装された OGC SQL 標準ST_関数。
- コア Mosaic 関数内での Spark コード生成の実装による高性能。
- Ordnance Survey (ブログ投稿)と共同開発したアプローチを使用して、ポイント イン ポリゴン結合を実行するための最適化。
- Scala、SQL、および Python API の選択。


`spark.read.json` が求めるファイルは一般的な JSON ファイルではないことに注意してください。各行は別個の自己内包の有効な JSON オブジェクトでなければなりません。これは [JSON Lines](https://jsonlines.org/) や [Newline-Delimited JSON](http://ndjson.org/) とも呼ばれます。


- https://databrickslabs.github.io/mosaic/index.html
- https://databrickslabs.github.io/mosaic/usage/quickstart.html

In [0]:
from pyspark.sql.functions import *

In [0]:
from mosaic import enable_mosaic
enable_mosaic(spark, dbutils)

In [0]:
from mosaic import st_point

lons = [-80., -80., -70., -70., -80.]
lats = [ 35.,  45.,  45.,  35.,  35.]

bounds_df = (
  spark
  .createDataFrame({"lon": lon, "lat": lat} for lon, lat in zip(lons, lats))
  .coalesce(1)
  .withColumn("point_geom", st_point("lon", "lat"))
)
bounds_df.show()

## JSON ファイルのロード

In [0]:
from mosaic import st_geomfromgeojson

geoJsonDF = (
  spark.read.format("json")
  .load("/mnt/testblob/geojson/niigata_cropfiled_geojsonl.json")
  .withColumn("geometry", st_geomfromgeojson(to_json(col("geometry"))))
  .select("properties.*", "geometry")
  .drop("shape_area", "shape_leng")
)

geoJsonDF.show()

## データの取得
PySparkでは、アクションを使用してDataFrame `show()` の先頭/最初の N (5,10,100 ..) 行を取得し、それらをコンソールまたはログに表示できます。行のリストとしての先頭と最後の n 行 (Scala の場合は Array[Row])。Spark アクションは Spark Driver に結果を取得するため、大きなデータセットを抽出するときは十分に注意する必要があります。

https://sparkbyexamples.com/spark/show-top-n-rows-in-spark-pyspark/

In [0]:
geoJsonDF.take(5)

In [0]:
geoJsonDF.count()

In [0]:
# RDD のパーティション数を返します
geoJsonDF.rdd.getNumPartitions()

## Mosaic による地理空間関数
Mosaic は、ジオメトリのプロパティを抽出するための多くの関数を提供します。Polygon ジオメトリに関連するものを次に示します。

In [0]:
from mosaic import st_area, st_length
(
  geoJsonDF
  .withColumn("calculatedArea", abs(st_area("geometry")))
  .withColumn("calculatedLength", st_length("geometry"))
  .select("geometry", "calculatedArea", "calculatedLength")
).show()